In [133]:
%%writefile create_data.py

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# jupyter notebook --execute <notebook>   - для запуска с терминала

dictdb = {}
for i in range(3):
    xs = np.linspace(1,10,100)
    ys = xs + np.random.random(100)*3-1
    dictdb['df'+str(i)] = pd.DataFrame({'x_cors':xs, 'y_cors':ys})

# шумные даные
xs = np.linspace(1,10,100)
ys = xs + np.random.random(100)*3-1
ys[50:60] *= 2
df_with_noise = pd.DataFrame({'x_cors':xs, 'y_cors':ys})
dictdb['df3'] = df_with_noise

model = LinearRegression()
model.fit(xs.reshape(-1, 1), ys)
r_sq = model.score(xs.reshape(-1, 1), y)
print('coefficient of determination:', r_sq)

Overwriting create_data.py


### Test

In [134]:
%%writefile test_emissions.py
from create_data import dictdb
import pytest
from scipy import stats
import numpy as np

def test_emissions():
  result = ''
  for df in dictdb.keys():
      z = stats.zscore(dictdb[df])
      dictdb[df][(np.abs(z) > 1.2).any(axis = 1)].head() # оставляем данные о 70%
    
      z_mask = (np.abs(z) < 1.2).all(axis = 1)
    
      new_df = dictdb[df][z_mask].copy()
      if dictdb[df][['x_cors', 'y_cors']].corr().loc['x_cors', 'y_cors'] < new_df[['x_cors', 'y_cors']].corr().loc['x_cors', 'y_cors']:
            result += f'Данные {df} имеют шумы и могут быть улучшены\n'
  assert  not result, result

      

Overwriting test_emissions.py


In [ ]:
! pytest test_emissions.py # верно, 3ый был с шумами

============================= test session starts ==============================
platform linux -- Python 3.11.6, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/sonikx/Documents
plugins: anyio-4.3.0
collecting ... 